### 1 - (A) , (B)



In [ ]:
! pip install transformers datasets evaluate

In [2]:
from huggingface_hub import notebook_login
notebook_login()

In [117]:
from datasets import load_dataset
ds = load_dataset("KaungHtetCho/Harry_Potter_LSTM")

In [118]:
train = ds['test'][0]['text']
print(train)
print(type(train))

Harry Potter and the Deathly Hallows By J. K. Rowling
<class 'str'>


In [119]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

# for reproducibility
torch.manual_seed(100)

In [125]:
# Dictionary
sample_sentences = [train]
char_set = list(set(''.join(sample_sentences)))
dic = {c: i for i, c in enumerate(char_set)}

# Parameters
dic_size = len(dic)
input_size = dic_size
hidden_size = dic_size * 2
output_size = dic_size

# Dataset setting
input_batch = []
target_batch = []

for sentence in sample_sentences:
  x_data = [dic[c] for c in sentence[:-1]]
  x_one_hot = [np.eye(dic_size)[x] for x in x_data]
  y_data = [dic[c] for c in sentence[1:]]
  input_batch.append(x_one_hot)
  target_batch.append(y_data)

# To torch tensors
X = torch.FloatTensor(np.array(input_batch))
Y = torch.LongTensor(np.array(target_batch))

print(X.shape)
print(Y.shape)

torch.Size([1, 52, 23])
torch.Size([1, 52])


In [126]:
# Model
class Custom_RNN(torch.nn.Module):
  def __init__(self, input_dim, hidden_dim, output_dim, layers):
    super(Custom_RNN, self).__init__()
    self.rnn = torch.nn.RNN(input_dim, hidden_dim, num_layers=layers)
    self.fc = torch.nn.Linear(hidden_dim, output_dim, bias=True)

  def forward(self, x):
    x, _status = self.rnn(x)
    x = self.fc(x)
    return x

learning_rate = 0.05
training_epochs = 100
model_rnn = Custom_RNN(input_size, hidden_size, output_size, 2)

In [127]:
# define cost/loss & optimizer
criterion = nn.CrossEntropyLoss()    # Softmax
optimizer = optim.Adam(model_rnn.parameters(), lr=learning_rate)

# train
for epoch in range(training_epochs):
  optimizer.zero_grad()
  outputs = model_rnn(X)
  loss = criterion(outputs.reshape(-1, dic_size), Y.reshape(-1))
  loss.backward()
  optimizer.step()
  if epoch % 10 == 9:
    print('epoch: ',epoch, 'loss: ', loss.item())

result = outputs.data.numpy().argmax(axis=2)
for sentence in result:
  print(''.join([char_set[c] for c in np.squeeze(sentence)]))


epoch:  9 loss:  0.9847201108932495
epoch:  19 loss:  0.9358469843864441
epoch:  29 loss:  0.9222496747970581
epoch:  39 loss:  0.9188047051429749
epoch:  49 loss:  0.9178280234336853
epoch:  59 loss:  0.9174587726593018
epoch:  69 loss:  0.9172264337539673
epoch:  79 loss:  0.9171422719955444
epoch:  89 loss:  0.9172341823577881
epoch:  99 loss:  0.9184433221817017
anrr DowhhrrDng DhlrDernhll Danllwl Dy D. D. Dowllng


2-(A) (B) (C) (D)

In [128]:
from datasets import load_dataset

ds_movie = load_dataset("sepidmnorozy/Korean_sentiment")

In [129]:
ds_movie['test'][0:3]['text']

['이 영화는 아름다운 영상미로 쓰레기같은 내용을 감추고 멋진 영화로 위장을 하고있다. 나는 이 영화가 만들다가 만 영화라고 생각한다.',
 '워메 ㅋㅋ 아이언맨이 여기나오는줄은 ㅋㅋ반전이네요',
 '여자 진짜 개귀엽다 ㅋㅋㅋㅋ']

In [131]:
import evaluate

accuracy = evaluate.load('accuracy')

import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [132]:
model1_name = 'jzonthemtn/distilbert-imdb'
model2_name = 'lvwerra/distilbert-imdb'


from transformers import AutoModelForSequenceClassification
model1 = AutoModelForSequenceClassification.from_pretrained(model1_name)
model2 = AutoModelForSequenceClassification.from_pretrained(model2_name)

In [133]:
classifier1 = pipeline('sentiment-analysis', model=model1_name, device=0)
classifier2 = pipeline('sentiment-analysis', model=model2_name, device=0)

In [134]:
average_accuracy_1 = 0.0 # 모델 1의 평균 정확도
average_accuracy_2 = 0.0 # 모델 2의 평균 정확도

for repeat in range(0, 3):
  test_list = ds_movie['test']
  test_sample_size = 100
  rng = np.random.default_rng()
  random_indices = rng.choice(len(test_list), size=test_sample_size, replace=False)
  sampled_data = test_list.select(random_indices)
  model_info = ""
  for i in range(1, 3):
    if i == 1:
      model = model1
      classifier = classifier1
      model_info = "model1"

    elif i==2:
      model = model2
      classifier = classifier2
      model_info = "model2"

    test_predictions = classifier(sampled_data['text'], max_length=512, truncation=True)
    predicted_labels = [label['label'] for label in test_predictions]
    mapped_labels = [model.config.label2id[label] for label in predicted_labels]
    reference_labels = sampled_data['label']
    print(f"Model: {model_info}")
    print(accuracy.compute(predictions=mapped_labels, references=reference_labels))
    print(reference_labels)
    print(mapped_labels)
    print()
    if i == 1:
      average_accuracy_1 += (accuracy.compute(predictions=mapped_labels, references=reference_labels)['accuracy'] / 3)
    elif i==2:
      average_accuracy_2 += (accuracy.compute(predictions=mapped_labels, references=reference_labels)['accuracy'] / 3)

Model: model1
{'accuracy': 0.54}
[1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1]

Model: model2
{'accuracy': 0.57}
[1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [135]:
print(f'모델 1의 평균 성능: {average_accuracy_1}')
print(f'모델 2의 평균 성능: {average_accuracy_2}')

if average_accuracy_1 > average_accuracy_2:
  print('모델 1이 모델 2 보다 평균적으로 우수합니다.')
elif average_accuracy_1 < average_accuracy_2:
  print('모델 2가 모델 1 보다 평균적으로 우수합니다.')
else:
  print('모델 1와 모델 2의 성능은 평균적으로 동일합니다.')

모델 1의 평균 성능: 0.53
모델 2의 평균 성능: 0.5233333333333333
모델 1이 모델 2 보다 평균적으로 우수합니다.
